In [ ]:
from logic.text_processing import TextProcessing
from logic.lexical_vectorizer import LexicalVectorizer
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from logic.stratified_sampling import random_stratified_sampling
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion

from root import DIR_INPUT

In [ ]:
text_procesing = TextProcessing(lang='es')

## **Preprocesamiento**

### Descripción del contenido de las columnas

- Id : Identificación unica del tweet
- Date : Tiempo universal coordinado (UTC) en el que el tweet fue creado
- Tweet : Texto en formato UTF8 de la actualización de estado
- Event : Indica si un tweet promueve o no la homofobia

In [ ]:
# se cargan los datos
df = pd.read_csv('{0}{1}.csv'.format(DIR_INPUT, 'homofobia'), sep=',')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
#verificacion de valores faltantes por columna
df.isna().sum()

In [ ]:
# estadisticos categoricos para la columna objetivo 
df['Event'] = df['Event'].astype(str)
df['Event'].describe()

### gráficas y distribuciones de frecuencia

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(15,8))
pos = np.arange(len(df['Event'].unique()[:2]))
bar = sns.countplot(ax=axes[0],x='Event',data=df ,order = df['Event'].value_counts().index)
axes[0].tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=True)
for p in bar.patches:
    bar.annotate(p.get_height(), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
for pos in ['right', 'top', 'bottom', 'left']:
    axes[0].spines[pos].set_visible(False)
bar.set_ylabel('')
bar.set_xlabel('Evento', size=13)
bar.set_title('Conteo de tweets por evento: 1 si hay homofobia 0 si no hay', size=15)
pie = axes[1].pie(x=df['Event'].value_counts(),labels = df['Event'].unique()[:2],autopct='%.0f%%')
_=axes[1].set_title('Distribución de los eventos', size=15)

### Aplicamos el metodo transformer de la clase TextProcesing

In [ ]:

df['Clean_tweet'] = [text_procesing.transformer(row) for row in df['Tweet'].tolist()]
df

### Dividimos el dataset usando muestreo aleatorio estratificado

In [ ]:
x_train, y_train, x_test, y_test = random_stratified_sampling(df,y='Event',test_size=0.2)

In [ ]:
# verificacion del balanceo de las clases tanto para las pruebas como para el entrenamiento
y_test.value_counts()

In [ ]:
y_train.value_counts()

In [ ]:
x_train

## **Extraccion de caracteristicas**

### Usaremos TF-IDF y vectorizacion lexica a traves de union de caracteristicas

In [ ]:
tfidf_vector = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
lexical_vectorizer = LexicalVectorizer(lang='es', text_processing=text_procesing)

In [ ]:
features_union = FeatureUnion([
            ('tfidf_vector', tfidf_vector),
            ('lexical_vector', lexical_vectorizer)
        ])

In [ ]:
features_union.fit(x_train['Clean_tweet'])